In [14]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object
import detectron.utils.vis as vis_utils
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

detector= 'Object365_Detector' #'HF_Detector' # 
visPath = '/home/chnxi/Detectron/backfill/visualizations/{}/'.format(detector)
cls_thrsh_file = '/home/chnxi/Detectron/backfill/{}/class_thresholds_at_pr.pkl'.format(detector)
cls_thrsh_dict = load_object(cls_thrsh_file)

dataset = 'FashionV2_train' # {'HF', 'OpenImage'}

In [ ]:
if detector == 'HF_Detector':
    #bboxFile = '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/FashionOnOIVal_VOC.json
    #imgPath = '/media/data/chnxi/OpenImage/validation_images/'
    if dataset == 'FashionV2_train':
        bboxFile = '/home/chnxi/Detectron/backfill/HF_Detector/FashionV2_train/test/FashionV2_train/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/FashionV2/Images/'
    elif dataset == 'OpenImage':
        #bboxFile = '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/_FashionV2_OpenImageTrain.json' #sys.argv[1]
        bboxFile = '/home/chnxi/Detectron/backfill/HF_Detector/OpenImage_train/test/OpenImage_train/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/OpenImage/train_images/'
    elif dataset == 'coco_2014_valminusminival':
        bboxFile = '/home/chnxi//Detectron/backfill/HF_Detector/coco_trainval/test/coco_2014_valminusminival/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/coco/val2014/'
elif detector == 'Object365_Detector':
    if dataset == 'FashionV2_train':
        bboxFile = '/home/chnxi/Detectron/backfill/Object365_Detector/FashionV2_train/test/FashionV2_train/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/FashionV2/Images/'
    elif dataset == 'OpenImage':
        #bboxFile = '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/_FashionV2_OpenImageTrain.json' #sys.argv[1]
        bboxFile = '/home/chnxi/Detectron/backfill/Object365_Detector/OpenImage_train/test/OpenImage_train/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/OpenImage/train_images/'
    elif dataset == 'coco_2014_valminusminival':
        bboxFile = '/home/chnxi//Detectron/backfill/Object365_Detector/coco_trainval/test/coco_2014_valminusminival/retinanet/detections.pkl' #sys.argv[1]
        imgPath = '/media/data/chnxi/coco/val2014/'
    


In [ ]:
visFolder = osp.join(visPath, dataset)
if not osp.isdir(visFolder):
    os.makedirs(visFolder)
    
ds = JsonDataset(dataset)
roidb = ds.get_roidb()
    
print("loading {}".format(bboxFile))
dets = load_object(bboxFile)
print ("loaded")
all_boxes = dets['all_boxes']

In [2]:
cls_thrsh_file = '/home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl'
cls_thrsh_dict = load_object(cls_thrsh_file)
print (cls_thrsh_dict.keys())
imgNum = len(roidb)
classes = cls_thrsh_dict['classes']

print(len(cls_thrsh_dict['prec_at'][0.9]))
print (len(classes))

print(roidb[0])

dict_keys(['prec_at', 'rec_at', 'classes', 'cls_aps'])
58
58
{'height': 300, 'width': 300, 'id': 0, 'dataset': <detectron.datasets.json_dataset.JsonDataset object at 0x7f512c980198>, 'image': '/media/data/chnxi//FashionV2/Images/b17ed5bd46400f89300cc903349e4c3e.jpg', 'flipped': False, 'has_visible_keypoints': False, 'boxes': array([], shape=(0, 4), dtype=float32), 'segms': [], 'gt_classes': array([], dtype=int32), 'seg_areas': array([], dtype=float32), 'gt_overlaps': <0x83 sparse matrix of type '<class 'numpy.float32'>'
	with 0 stored elements in Compressed Sparse Row format>, 'is_crowd': array([], dtype=bool), 'box_to_gt_ind_map': array([], dtype=int32), 'max_classes': array([], dtype=int64), 'max_overlaps': array([], dtype=float32)}


In [3]:
# cls_prthrsh_dict = {}
# for cid, cls in enumerate(classes):
#     cls_thrshs = {'prec_at_0.9': cls_thrsh_dict['prec_at'][0.9][cid], 
#                   'prec_at_0.8': cls_thrsh_dict['prec_at'][0.8][cid],
#                   'rec_at_0.5' : cls_thrsh_dict['rec_at'][0.5][cid],
#                   'rec_at_0.7' : cls_thrsh_dict['rec_at'][0.7][cid]}
#     cls_prthrsh_dict[cid] = cls_thrshs

new_cls_thrsh_dict = {'prec_at_0.9': cls_thrsh_dict['prec_at'][0.9], 
                  'prec_at_0.8': cls_thrsh_dict['prec_at'][0.8],
                  'rec_at_0.5' : cls_thrsh_dict['rec_at'][0.5],
                  'rec_at_0.7' : cls_thrsh_dict['rec_at'][0.7]}
    
print (new_cls_thrsh_dict['prec_at_0.9'])

[0.65890813 0.40394038 0.5774321  0.77713573 0.60711682 0.67322326
 0.52851528 0.46174601 0.61438024 0.64767092 0.81435126 0.53951627
 0.57961816 0.60969418 0.62125641 0.5989415  0.58761984 0.63688034
 0.59885871 0.5786556  0.66398668 0.59098428 0.64770222 0.60134423
 0.71601284 0.78756082 0.59866714 0.53077555 0.55953896 0.68613309
 0.84925234 0.65713745 0.65950334 0.56061822 0.70493436 0.56271166
 0.70769304 0.77569616 0.63649017 0.65164274 0.57107413 0.57096714
 0.59916556 0.6672796  0.57261157 0.66317564 0.66976112 0.59649551
 0.57466114 0.69485384 0.66893601 0.78391629 0.78418601 0.35948369
 0.55339015 0.76437533 0.76778102 0.50362223]


In [10]:
# print (new_cls_thrsh_dict['prec_at_0.9'])
import os
import os.path as osp
visPath = '../visualizations/'
if not osp.isdir(visPath):
    os.makedirs(visPath)
    
keylist = ['prec_at_0.9', 'prec_at_0.8', 'rec_at_0.5', 'rec_at_0.7']
color_list = ['g', 'r', 'b', 'm']

def visImageClsThrshs(plt, splot, im, imgBoxes, ecolor='m', title='', showCateg=False):
    #plt.cla()
    fx, fy, fi = splot
    plt.subplot(fx, fy, fi)
    plt.imshow(im)
    plt.axis('off')
    plt.title(title)
    ax = plt.gca()
    for cid, cls_boxes in enumerate(imgBoxes):
        for obj in cls_boxes:
            #print obj
            bbox = obj[:4]
            score = obj[-1]
            categ = classes[cid]            
            #ecolor = 'm' #if gid == jmax else gtcolor
            #print "{}: bbox = {}".format(categ, bbox)
            ax.add_patch(
                plt.Rectangle((bbox[0], bbox[1]),
                              bbox[2] - bbox[0],
                              bbox[3] - bbox[1], fill=False,
                              edgecolor=ecolor, linestyle='-', linewidth=0.5))
            if showCateg:
                ax.text(bbox[0], bbox[3], '{}:{:.1f}'.format(categ, score), bbox={'facecolor': ecolor, 'alpha': 0.5})
        plt.draw()

import matplotlib.pyplot as plt
from PIL import Image
saveVis = False
showVis = True
        
import random
visNum = 1
imgIds = random.sample(range(imgNum), visNum)
print(imgIds)
    
def id_or_index(ix, val):
    if len(val) == 0:
        return val
    else:
        return val[ix]
        
for ix, entry in enumerate(roidb):
    imgFileName = entry['image']
    imgBaseName = osp.basename(imgFileName)
    print (imgBaseName)
    print (imgFileName)
    im = Image.open(imgFileName)
    img_boxes = [
        id_or_index(ix, cls_k_boxes) for cls_k_boxes in all_boxes
    ]
    img_boxes = img_boxes[1:] # remove background boxes
        
    f, axarr = plt.subplots(1, 4, sharey=True, figsize=(20, 20))
    plt.cla()
    fid = 0
    total_box_num = 0
    for fid, key in enumerate(keylist): #['prec_at_0.9']: #new_cls_thrsh_dict:
        print (key)
        ax = axarr[fid]
        thresholds = new_cls_thrsh_dict[key]
        img_boxes_thrshed_num = 0
        img_boxes_thrshed = []
        for cid, cls_boxes in enumerate(img_boxes):
            #print (cls_boxes.shape)
            cls_boxes_thrshed = [bbox for bbox in cls_boxes if bbox[-1] > thresholds[cid]]
            cls_boxes_num = len(cls_boxes_thrshed)
            img_boxes_thrshed.append(cls_boxes_thrshed)
            img_boxes_thrshed_num += cls_boxes_num
        print(img_boxes_thrshed_num)
        total_box_num += img_boxes_thrshed_num
        visImageClsThrshs(plt, (1,4, fid+1), im, img_boxes_thrshed, ecolor=color_list[fid], title=key, showCateg=True)
        fid += 1
    if showVis:
        plt.draw()
        plt.show();
    if total_box_num > 0:
        plt.savefig(osp.join(visFolder,imgBaseName), bbox_inches='tight',pad_inches = 0)
    #plt.close()
print ("Done!")

[26682]
b17ed5bd46400f89300cc903349e4c3e.jpg
/media/data/chnxi//FashionV2/Images/b17ed5bd46400f89300cc903349e4c3e.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
1585d300f5bc46e95cab909db8b757c2.jpg
/media/data/chnxi//FashionV2/Images/1585d300f5bc46e95cab909db8b757c2.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
b6ff4ab255d81f4fd96a45a35957d3a8.jpg
/media/data/chnxi//FashionV2/Images/b6ff4ab255d81f4fd96a45a35957d3a8.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
1574ef90e1d27a8df78e6f811b6aa875.jpg
/media/data/chnxi//FashionV2/Images/1574ef90e1d27a8df78e6f811b6aa875.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
d7aef289adc41ae40c76e08ad67acd35.jpg
/media/data/chnxi//FashionV2/Images/d7aef289adc41ae40c76e08ad67acd35.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
061ad283a806ad5bbb0b42f914213a9a.jpg
/media/data/chnxi//FashionV2/Images/061ad283a806ad5bbb0b42f914213a9a.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
3
9700ae4f

/home/chnxi/miniconda2/envs/detectron/lib/python3.6/site-packages/matplotlib-3.0.3-py3.6-linux-x86_64.egg/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
dd4268b9074ab336dc6c5bcdae4b4b56.jpg
/media/data/chnxi//FashionV2/Images/dd4268b9074ab336dc6c5bcdae4b4b56.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
ba9d6f00f78253322fed6642b1aae1b7.jpg
/media/data/chnxi//FashionV2/Images/ba9d6f00f78253322fed6642b1aae1b7.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
2
c9accd7a9c4a155ef93d3259fe0c0feb.jpg
/media/data/chnxi//FashionV2/Images/c9accd7a9c4a155ef93d3259fe0c0feb.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
39747f9bfc212b072ed851e1635f3a8d.jpg
/media/data/chnxi//FashionV2/Images/39747f9bfc212b072ed851e1635f3a8d.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
d07992e0ea4dc75a03b84443bc72557e.jpg
/media/data/chnxi//FashionV2/Images/d07992e0ea4dc75a03b84443bc72557e.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
037cd0ef054e77fdb7b844b2c269f863.jpg
/media/data/chnxi//FashionV2/Images/037cd0ef054e77fdb7b844b2c269f863.jpg
prec_at_0.9
0
prec_at_0.8
0
re

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
0b116240042179c1c85f4294181dd6b1.jpg
/media/data/chnxi//FashionV2/Images/0b116240042179c1c85f4294181dd6b1.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
46b4b692a65120db54fccb012c1441cd.jpg
/media/data/chnxi//FashionV2/Images/46b4b692a65120db54fccb012c1441cd.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
6
154e3fef0fb4d30a89c2e3dd8bf536cc.jpg
/media/data/chnxi//FashionV2/Images/154e3fef0fb4d30a89c2e3dd8bf536cc.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
a3aec51c7177ab480d1a3d7bf526aacc.jpg
/media/data/chnxi//FashionV2/Images/a3aec51c7177ab480d1a3d7bf526aacc.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
93a64174416b4e131d4a299e1791d77a.jpg
/media/data/chnxi//FashionV2/Images/93a64174416b4e131d4a299e1791d77a.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
8
cf42abdbf3f821186ed508c0c412e232.jpg
/media/data/chnxi//FashionV2/Images/cf42abdbf3f821186ed508c0c412e232.jpg
prec_at_0.9
0
prec_at_0.8
0
re

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
2
fd17b484610ae1f2f9b419468fe56c38.jpg
/media/data/chnxi//FashionV2/Images/fd17b484610ae1f2f9b419468fe56c38.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
cdb4d8a4a7a32653734c6fdd1ac66187.jpg
/media/data/chnxi//FashionV2/Images/cdb4d8a4a7a32653734c6fdd1ac66187.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
6e8521f4974192fdfccc1ce81544ff2b.jpg
/media/data/chnxi//FashionV2/Images/6e8521f4974192fdfccc1ce81544ff2b.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
3925ba7f2111d5525513c08a34b605fb.jpg
/media/data/chnxi//FashionV2/Images/3925ba7f2111d5525513c08a34b605fb.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
4
8f76cda71d160cfa9431a3da215119ea.jpg
/media/data/chnxi//FashionV2/Images/8f76cda71d160cfa9431a3da215119ea.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
9b12599e531457bae1d45406fe6ca46d.jpg
/media/data/chnxi//FashionV2/Images/9b12599e531457bae1d45406fe6ca46d.jpg
prec_at_0.9
0
prec_at_0.8
0
re

KeyboardInterrupt: 